In [8]:
""" 6-1 Milestone: Dashboard Data Visualizations by Paul Kenaga """
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
from PIL import Image


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


# Import CRUD module to access CRUD functionality
from CRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################

# Set values for user authenticatoin and instaniate AnimalShelter object
username = 'aacuser'
password = 'bloop'
shelter = AnimalShelter(username, password)

# Set value for image path
image_path = 'assets/Logo2.png'

# Call read method to return cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

# Use Pillow to read the the image
pil_img =Image.open(image_path)

#########################
# Dashboard Layout / View
#########################

# Name the app
app = JupyterDash('PaulOliverK')

# Create layout to describe what the app looks like
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Hr(),
    html.Center(html.B(html.H1('CS340 Interactive Dashboard'), style={'flex': 1})),    #  Title - bold
    html.Center(html.A([html.Img(src=pil_img)], href='www.snhu.edu')),    # Add client Logo with URL anchor tag - centered  
    html.Center(html.H6('Created By Paul Kenaga', style={'color' : '#68228b'})),    #  Unique Identifier - purple,
    html.Hr(),
    html.B(html.Label('Filter Rescue Type:')),    # Label for radio items - bold
    #Create Rescue Radio Items - Value set to 'All' so that no radio item is pre-selected in beginning 
    html.Div([dcc.RadioItems(id = 'radio-id',
                                      options = [dict(label = 'Water', value = 'water'),
                                                 dict(label = 'Mountain or Wilderness', value = 'mw'),
                                                 dict(label = 'Disaster or Individual Tracking', value = 'dit'),
                                                 dict(label = 'Reset', value = 'reset')],
                                      value = 'All')]),        
    # Create interactive data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {'name': i, 'id': i, 'deletable': False, 'selectable': True} for i in df.columns],
        data=df.to_dict('records'),    # Import data
        # Make table width the size of the page and allow horizontal scroll for overfloww
        style_table={'overflowX': 'auto'},    
        style_data={
            'color': 'black',
            'backgroundColor': 'white',
            'border': '2px solid black'},    # Added solid black border 
        style_header={
            'backgroundColor': 'rgb(210, 210, 210)',    # Set header background to grey
            'color': 'black',
            'fontWeight': 'bold',    # Make header bold
            'border': '2px solid black'},    # Added solid black border 
        editable=False,    # Table is not editable
        filter_action='native',    # Display filtering UI: logic handled by table
        sort_action='native',    # Display sorting UI: logic handled by table
        sort_mode='mulit',    # Allow sorting across multiple columns
        column_selectable=False,    # Columns are not selectable
        row_selectable=False,    # Rows are not selectable  
        row_deletable=False,    # Rows are not deletable
        selected_columns=[],    # Empty because column_selectable=False
        selected_rows=[],    # Empty because row_selectable=False
        page_action='native',    # Pass data to the table up-front
        page_current=0,     # Set first page to 0
        page_size=5,    # Set number of rows to 5
    ),
    html.Br(),
    html.Hr(),
    # Create Drop-Down to change pie chart view between outcome and breed
    html.Div([
        html.B(html.Label(['Filter Pie Chart:'])),    # Label for drop-down - bold 
        dcc.Dropdown(
            id='dropdown-id',
            options=[
                     {'label': 'Breed', 'value': 'breed'},
                     {'label': 'Outcome', 'value': 'outcome_type'},      
            ],
            value='outcome_type',
            multi=False,
            clearable=False,
            style={'width': '50%',
                   'flex': 1,    # Flex-grow
                   }
        ),
    ]),
    # Allow pie chart and geolocation chart to be on same row
    html.Div(className='row',
            style={'display': 'flex'},
                children=[
                    #Create Pie chart
                    html.Div(
                        id='pie-id',
                        className='col s12 m6'),
                    # Create geolocation chart
                     html.Div(
                        id='map-id',
                        className='col s12 m6',
                        ), 
                ]),
])
#############################################
# Interaction Between Components / Controller
#############################################

''' This callback interprets which radio item is selected and outputs what data to show
'''
@app.callback(Output('datatable-id', 'data'),
              [Input('radio-id', 'value')])
def update_graph(value):
    # Show all records (beginning and if 'reset' radio item selected)
    if value == 'All' or value == 'reset':
        df=pd.DataFrame.from_records(shelter.read({}))
        
    # Water rescue compound query
    elif value == 'water':
        df=pd.DataFrame(list(shelter.read({'$and' : [
                                            { '$or' : [ { 'breed' : 'Labrador Retriever Mix' }, 
                                                        { 'breed' : 'Chesapeake Bay Retriever' }, 
                                                        { 'breed' : 'Newfoundland' } ] },
                                            { 'sex_upon_outcome' : 'Intact Female' },
                                            { 'age_upon_outcome_in_weeks' :  { '$gte' : 26 } },
                                            { 'age_upon_outcome_in_weeks' :  { '$lte' : 156 } }
        ]})))
    
    # Mountain or wilderness rescue compound query
    elif value == 'mw':
        df=pd.DataFrame(list(shelter.read({'$and' : [
                                            { '$or' : [ { 'breed' : 'German Shepherd' }, 
                                                        { 'breed' : 'Alaskan Malamute' }, 
                                                        { 'breed' : 'Old English Sheepdog' }, 
                                                        { 'breed' : 'Siberian Husky' }, 
                                                        { 'breed' : 'Rottweiler' } ] },
                                            { 'sex_upon_outcome' : 'Intact Male' },
                                            { 'age_upon_outcome_in_weeks' :  { '$gte' : 26 } },
                                            { 'age_upon_outcome_in_weeks' :  { '$lte' : 156 } }
        ]})))
    # Disaster or individual tracking rescue compound query
    elif value == 'dit':
        df=pd.DataFrame(list(shelter.read({'$and' : [
                                            { '$or' : [ { 'breed' : 'Doberman Pinscher' }, 
                                                        { 'breed' : 'German Shepherd' }, 
                                                        { 'breed' : 'Golden Retriever' }, 
                                                        { 'breed' : 'Bloodhound' }, 
                                                        { 'breed' : 'Rottweiler' } ] },
                                            { 'sex_upon_outcome' : 'Intact Male' },
                                            { 'age_upon_outcome_in_weeks' :  { '$gte' : 20 } },
                                            { 'age_upon_outcome_in_weeks' :  { '$lte' : 300 } }
        ]})))
    return df.to_dict('records')


''' This callback will highlight a cell on the data table when the user selects it
'''
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')
    ])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


''' This callback will display a geolocation chart underneath the interactive data table
    Map is centered at longitude & latitude of Austin
    Width/height of geo chart set to 50% of viewport
'''
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')])
def update_map(viewData):
#Created geolocation chart
    dff = pd.DataFrame.from_dict(viewData)    # Retrieve dataframe
    row_count = dff.shape[0]    # Get row count to determine how many markers to show
    # Show geolocation chart w/o markers if 0 rows in dataframe
    if row_count == 0:
        return [
            dl.Map(style={'width':'50vh','height': '50vh'}, center=[(30.26759),(-97.74299)], zoom=8, children=[
                dl.TileLayer(id="base-layer-id")])]
                
    # Show 1 marker if only 1 row in dataframe
    if row_count == 1:
        return [
            dl.Map(style={'width':'50vh','height': '50vh'}, center=[(30.26759),(-97.74299)], zoom=8, children=[
                dl.TileLayer(id="base-layer-id"),

                # Marker 1 placed at longitude & latitude of first animal in the table
                dl.Marker(position=[(dff.iloc[0,13]),(dff.iloc[0,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[0,3]), ': ', (dff.iloc[0,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[0,9])),
                        html.P(['Animal ID: ', dff.iloc[0,2], html.Br(), 'Outcome: ', dff.iloc[0,11],
                               html.Br(), 'Details: ', dff.iloc[0,10],
                               ])
                    ])
                ])
            ])
        ]
    
    # Show 2 markers if only 2 rows in dataframe
    if row_count == 2:
        return [
            dl.Map(style={'width':'50vh','height': '50vh'}, center=[(30.26759),(-97.74299)], zoom=8, children=[
                dl.TileLayer(id="base-layer-id"),

                # Marker 1 placed at longitude & latitude of first animal in the table
                dl.Marker(position=[(dff.iloc[0,13]),(dff.iloc[0,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[0,3]), ': ', (dff.iloc[0,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[0,9])),
                        html.P(['Animal ID: ', dff.iloc[0,2], html.Br(), 'Outcome: ', dff.iloc[0,11],
                               html.Br(), 'Details: ', dff.iloc[0,10],
                               ])
                    ])
                ]),
                # Marker 2 placed at longitude & latitude of second animal in the table
                dl.Marker(position=[(dff.iloc[1,13]),(dff.iloc[1,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[1,3]), ': ', (dff.iloc[1,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[1,9])),
                        html.P(['Animal ID: ', dff.iloc[1,2], html.Br(), 'Outcome: ', dff.iloc[1,11],
                               html.Br(), 'Details: ', dff.iloc[1,10],
                               ])
                    ])
                ]),
            ])
        ]
    
    # Show 3 markers if only 3 rows in dataframe
    if row_count == 3:
        return [
            dl.Map(style={'width':'50vh','height': '50vh'}, center=[(30.26759),(-97.74299)], zoom=8, children=[
                dl.TileLayer(id="base-layer-id"),

                # Marker 1 placed at longitude & latitude of first animal in the table
                dl.Marker(position=[(dff.iloc[0,13]),(dff.iloc[0,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[0,3]), ': ', (dff.iloc[0,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[0,9])),
                        html.P(['Animal ID: ', dff.iloc[0,2], html.Br(), 'Outcome: ', dff.iloc[0,11],
                               html.Br(), 'Details: ', dff.iloc[0,10],
                               ])
                    ])
                ]),
                # Marker 2 placed at longitude & latitude of second animal in the table
                dl.Marker(position=[(dff.iloc[1,13]),(dff.iloc[1,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[1,3]), ': ', (dff.iloc[1,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[1,9])),
                        html.P(['Animal ID: ', dff.iloc[1,2], html.Br(), 'Outcome: ', dff.iloc[1,11],
                               html.Br(), 'Details: ', dff.iloc[1,10],
                               ])
                    ])
                ]),
                # Marker 3 placed at longitude & latitude of third animal in the table
                dl.Marker(position=[(dff.iloc[2,13]),(dff.iloc[2,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[2,3]), ': ', (dff.iloc[2,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[2,9])),
                        html.P(['Animal ID: ', dff.iloc[2,2], html.Br(), 'Outcome: ', dff.iloc[2,11],
                               html.Br(), 'Details: ', dff.iloc[2,10],
                               ])
                    ])
                ]),
            ])
        ]
    
    # Show 4 markers if only 4 rows in dataframe
    if row_count == 4:
        return [
            dl.Map(style={'width':'50vh','height': '50vh'}, center=[(30.26759),(-97.74299)], zoom=8, children=[
                dl.TileLayer(id="base-layer-id"),

                # Marker 1 placed at longitude & latitude of first animal in the table
                dl.Marker(position=[(dff.iloc[0,13]),(dff.iloc[0,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[0,3]), ': ', (dff.iloc[0,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[0,9])),
                        html.P(['Animal ID: ', dff.iloc[0,2], html.Br(), 'Outcome: ', dff.iloc[0,11],
                               html.Br(), 'Details: ', dff.iloc[0,10],
                               ])
                    ])
                ]),
                # Marker 2 placed at longitude & latitude of second animal in the table
                dl.Marker(position=[(dff.iloc[1,13]),(dff.iloc[1,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[1,3]), ': ', (dff.iloc[1,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[1,9])),
                        html.P(['Animal ID: ', dff.iloc[1,2], html.Br(), 'Outcome: ', dff.iloc[1,11],
                               html.Br(), 'Details: ', dff.iloc[1,10],
                               ])
                    ])
                ]),
                # Marker 3 placed at longitude & latitude of third animal in the table
                dl.Marker(position=[(dff.iloc[2,13]),(dff.iloc[2,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[2,3]), ': ', (dff.iloc[2,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[2,9])),
                        html.P(['Animal ID: ', dff.iloc[2,2], html.Br(), 'Outcome: ', dff.iloc[2,11],
                               html.Br(), 'Details: ', dff.iloc[2,10],
                               ])
                    ])
                ]),
                # Marker 4 placed at longitude & latitude of fourth animal in the table
                dl.Marker(position=[(dff.iloc[3,13]),(dff.iloc[3,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[3,3]), ': ', (dff.iloc[3,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[3,9])),
                        html.P(['Animal ID: ', dff.iloc[3,2], html.Br(), 'Outcome: ', dff.iloc[3,11],
                               html.Br(), 'Details: ', dff.iloc[3,10],
                               ])
                    ])
                ]),
            ])
        ]
    
    # Show 5 markers if there are 5 rows in dataframe
    if row_count == 5:
        return [
            dl.Map(style={'width':'50vh','height': '50vh'}, center=[(30.26759),(-97.74299)], zoom=8, children=[
                dl.TileLayer(id="base-layer-id"),

                # Marker 1 placed at longitude & latitude of first animal in the table
                dl.Marker(position=[(dff.iloc[0,13]),(dff.iloc[0,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[0,3]), ': ', (dff.iloc[0,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[0,9])),
                        html.P(['Animal ID: ', dff.iloc[0,2], html.Br(), 'Outcome: ', dff.iloc[0,11],
                               html.Br(), 'Details: ', dff.iloc[0,10],
                               ])
                    ])
                ]),
                # Marker 2 placed at longitude & latitude of second animal in the table
                dl.Marker(position=[(dff.iloc[1,13]),(dff.iloc[1,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[1,3]), ': ', (dff.iloc[1,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[1,9])),
                        html.P(['Animal ID: ', dff.iloc[1,2], html.Br(), 'Outcome: ', dff.iloc[1,11],
                               html.Br(), 'Details: ', dff.iloc[1,10],
                               ])
                    ])
                ]),
                # Marker 3 placed at longitude & latitude of third animal in the table
                dl.Marker(position=[(dff.iloc[2,13]),(dff.iloc[2,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[2,3]), ': ', (dff.iloc[2,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[2,9])),
                        html.P(['Animal ID: ', dff.iloc[2,2], html.Br(), 'Outcome: ', dff.iloc[2,11],
                               html.Br(), 'Details: ', dff.iloc[2,10],
                               ])
                    ])
                ]),
                # Marker 4 placed at longitude & latitude of fourth animal in the table
                dl.Marker(position=[(dff.iloc[3,13]),(dff.iloc[3,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[3,3]), ': ', (dff.iloc[3,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[3,9])),
                        html.P(['Animal ID: ', dff.iloc[3,2], html.Br(), 'Outcome: ', dff.iloc[3,11],
                               html.Br(), 'Details: ', dff.iloc[3,10],
                               ])
                    ])
                ]),
                # Marker 5 placed at longitude & latitude of fifth animal in the table
                dl.Marker(position=[(dff.iloc[4,13]),(dff.iloc[4,14])], children=[
                    # Tooltip displays animal_type & breed
                    dl.Tooltip([html.B(dff.iloc[4,3]), ': ', (dff.iloc[4,4])]),
                    # Popup displays animal_name, outcome_type, & outcome_subtype
                    dl.Popup([
                        html.Center(html.B(dff.iloc[4,9])),
                        html.P(['Animal ID: ', dff.iloc[4,2], html.Br(), 'Outcome: ', dff.iloc[4,11],
                               html.Br(), 'Details: ', dff.iloc[4,10],
                               ])
                    ])
                ])
            ])
        ]

    
'''
   This callback will display a pie chart next to the geolocation chart
   It uses input from the drop-down component and data table
   Width/height of pie chart set to 50% of viewport
'''
@app.callback(
    Output(component_id='pie-id', component_property='children'),
    [Input(component_id='dropdown-id', component_property='value'),
     Input('datatable-id', 'derived_viewport_data')
    ])
def update_pie(value, viewData):
    dff = pd.DataFrame.from_dict(viewData)    #Retrieve dataframe
    return [
            dcc.Graph(
                figure =px.pie(
                   data_frame=dff,
                   color_discrete_sequence= px.colors.sequential.Agsunset,   # Set sequential color scheme 
                   names=value,    # Set values for pie chart based on drop-down
                    ), style={'width':'50vh','height': '50vh'}    
            )
    ]

app